In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        100,            #  3rd last digit before decimal
        10,             #  2nd last
        1,              #  last
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

# ---

c1, c2, m1, m2 = var("c1, c2, m1, m2", positive=True)

c, m = var("c, m", positive=True)

K = Matrix([
[c1 + c2,  -c2],
[-c2,  c2],
])

M = Matrix([
[m1, 0],
[0, m2],
])

sub_list = [
    (m1,m),
    (m2,2*m),
    (c1,c),
    (c2,c),
]

# ξ = λ²
xi = var("xi")
w = var("omega")

pprint("\nK + λ² M:")
A = K + xi*M
pprint(A)

pprint("\nK + λ²M for (m1=m, m2=2m, c1=c, c2=c):")
A = A.subs(sub_list)
pprint(A)
# exit()
pprint("\nCharacteristic equation:")
eq = Eq(det(A))
pprint(eq)

sol_xi = solve(eq,xi)

# Init:
eigenfreqs = [0,0]
eigenvects = [Matrix([1,0]), Matrix([1,0])]

x1, x2 = var("x1, x2")
w = Matrix([x1, x2])

pprint("\n")
for i in range(len(sol_xi)):
    pprint("\n--- (Value, Vector)-Pair Number "+str(i+1)+" ---")
    pprint("\nValue λ² / (c/m):")
    xii = sol_xi[i]
    eigenfreqs[i] = sqrt(-xii)
    tmp = xii / (c/m)
    pprint(tmp)
    Ai = A.subs(xi,xii)
    pprint("\nVector:")
    eq = Eq(Ai*w,Matrix([0,0]))
    sol = solve(eq, x2)
    pprint(sol)
    tmp = sol[x2]
    tmp = tmp.subs(x1, 1)
    eigenvects[i][1] = tmp

pprint("\nEigenfrequencies / sqrt(c/m) and resp. Eigenvector for x1=1:")
for i in range(2):
    pprint("\n--- (Value, Vector)-Pair Number "+str(i+1)+" ---")
    tmp = eigenfreqs[i]
    tmp /= sqrt(c/m)
    pprint(tmp)
    tmp = iso_round(tmp,0.01)
    pprint(tmp)
    tmp = eigenvects[i]
    pprint(tmp)
    tmp = iso_round(tmp,0.01)
    pprint(tmp)

# K + λ² M:
# ⎡c₁ + c₂ + m₁⋅ξ     -c₂   ⎤
# ⎢                         ⎥
# ⎣     -c₂        c₂ + m₂⋅ξ⎦
#
# K + λ²M for (m1=m, m2=2m, c1=c, c2=c):
# ⎡2⋅c + m⋅ξ     -c    ⎤
# ⎢                    ⎥
# ⎣   -c      c + 2⋅m⋅ξ⎦
#
# Characteristic equation:
#    2
# - c  + (c + 2⋅m⋅ξ)⋅(2⋅c + m⋅ξ) = 0
#
#
# --- (Value, Vector)-Pair Number 1 ---
#
# Value λ² / (c/m):
#   5   √17
# - ─ + ───
#   4    4
#
# Vector:
# ⎧    x₁⋅(3 + √17)⎫
# ⎨x₂: ────────────⎬
# ⎩         4      ⎭
#
# --- (Value, Vector)-Pair Number 2 ---
#
# Value λ² / (c/m):
#   5   √17
# - ─ - ───
#   4    4
#
# Vector:
# ⎧    x₁⋅(-√17 + 3)⎫
# ⎨x₂: ─────────────⎬
# ⎩          4      ⎭
#
# Eigenfrequencies / sqrt(c/m) and resp. Eigenvector for x1=1:
#
# --- (Value, Vector)-Pair Number 1 ---
#   __________
# ╲╱ -√17 + 5
# ────────────
#      2
# 0.47
# ⎡   1   ⎤
# ⎢       ⎥
# ⎢3   √17⎥
# ⎢─ + ───⎥
# ⎣4    4 ⎦
# ⎡1.0 ⎤
# ⎢    ⎥
# ⎣1.78⎦
#
# --- (Value, Vector)-Pair Number 2 ---
#   _________
# ╲╱ √17 + 5
# ───────────
#      2
# 1.51
# ⎡    1    ⎤
# ⎢         ⎥
# ⎢  √17   3⎥
# ⎢- ─── + ─⎥
# ⎣   4    4⎦
# ⎡ 1.0 ⎤
# ⎢     ⎥
# ⎣-0.28⎦
